In [2]:
!pip install spacy

In [1]:
!pip install num2words

In [2]:
!pip install pandas

  Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\User\\anaconda3\\Lib\\site-packages\\numpy\\~0ibs\\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



In [3]:
pip install --upgrade numpy

Note: you may need to restart the kernel to use updated packages.


In [51]:
import pandas as pd
import spacy
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import contractions
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
from nltk.tokenize import TweetTokenizer
import regex
import nltk
from nltk.stem import SnowballStemmer
import demoji
import emoji
from sklearn.metrics import f1_score
demoji.download_codes()
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,accuracy_score
from sklearn.svm import LinearSVC
from num2words import num2words
from sklearn.feature_extraction.text import CountVectorizer

C:\Users\User\AppData\Local\Temp/ipykernel_9608/2120401968.py:17: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('whitespace')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading whitespace: Package 'whitespace' not found
[nltk_data]     in index
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
data = pd.read_csv('Training.tsv',sep='\t')

In [4]:
data.head(10)

,tweet_id,text,label
0,1382343793341575169,@IrvineWelsh I don’t know about you Irvine b...,0
1,1377631738692796417,https://t.co/T2aD3v6dpm I bet money if i went...,0
2,1386448010029240326,@JamesMelville My wife received a POSITIVE Cov...,0
3,1361342676340211717,"Out of the 180,000+ people who have had the tw...",0
4,1386757983254765569,My whole family is sick af and here I am now i...,0
5,1382001700853125122,@renfrew1962 @PeakePolly @J_Deliciouso I'm not...,0
6,1383272654212272136,"Test came back positive, no surprise. I have c...",1
7,1374479299047084035,My Pawpaw has been in the hospital a few days....,0
8,1354020426620547072,@MattHancock 4 people I know had covid and rec...,0
9,1362671045136809985,I’m going to sound like I have lost my marbles...,1


In [5]:
print(data.columns)

Index(['tweet_id', 'text', 'label'], dtype='object')


In [6]:
data['label'].value_counts()

label
0    6266
1    1334
Name: count, dtype: int64

# Preprocessing

In [7]:
#emot_object = emot.core.emot()
ps =PorterStemmer()
lemmatiser = WordNetLemmatizer()
english_stopwords = stopwords.words('english')
exclude = set(string.punctuation)
def preprocess(text):
  #text=demoji.findall(df['Text'])
  text = contractions.fix(text.lower(), slang=True)
  #text= re.sub(r'\d+', '', text)
  text = re.sub(r'\d+', lambda x: num2words(int(x.group(0))), text)
  text=re.sub(r'@\w+','',text)
  text=re.sub(r'$', '', text)
  text= re.sub(r'’','', text )  
  text=re.sub('<.*?>','',text)
  text=re.sub(r'http\S+', '', text)
  #text=emoji.demojize(text, delimiters=(" ", " "))
  text = ''.join(ch for ch in text if ch not in exclude)
  tokens = word_tokenize(text)
  #print("Tokens:", tokens)
  text = [t for t in tokens if t not in english_stopwords]
  text = " ".join(text)
  return text

In [8]:
import emoji
#import demoji

#demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

In [9]:
data['emo']=data["text"].apply(lambda x:emo(x))
data["clean_text"]=data['emo'].apply(lambda X: preprocess(X))
#added_data["clean_text"]

In [10]:
data.head()

,tweet_id,text,label,emo,clean_text
0,1382343793341575169,@IrvineWelsh I don’t know about you Irvine b...,0,@IrvineWelsh I don’t know about you Irvine b...,know irvine keep told covid exist therefore do...
1,1377631738692796417,https://t.co/T2aD3v6dpm I bet money if i went...,0,https://t.co/T2aD3v6dpm I bet money if i went...,bet money went n took covid test right going t...
2,1386448010029240326,@JamesMelville My wife received a POSITIVE Cov...,0,@JamesMelville My wife received a POSITIVE Cov...,wife received positive covid test result despi...
3,1361342676340211717,"Out of the 180,000+ people who have had the tw...",0,"Out of the 180,000+ people who have had the tw...",one hundred eightyzero people two vaccine shot...
4,1386757983254765569,My whole family is sick af and here I am now i...,0,My whole family is sick af and here I am now i...,whole family sick af hospital heart palpitatio...


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['clean_text'], data['label'], test_size=0.33, random_state=42)

In [12]:
data.tail(30)

,tweet_id,text,label,emo,clean_text
7570,1375556191900798982,"In 12 months, I only know of 3 people who test...",0,"In 12 months, I only know of 3 people who test...",twelve months know three people tested positiv...
7571,1386424325268557832,@anet2111 I live in a state with the populatio...,0,@anet2111 I live in a state with the populatio...,thousand one hundred eleven live state populat...
7572,1381642657429082112,"Yupppp, I've seen a few stroke patients in the...",0,"Yupppp, I've seen a few stroke patients in the...",yupppp seen stroke patients fortys fiftys reha...
7573,1377644044197752837,@EvelKneidel Actually my gym costs $10 a month...,0,@EvelKneidel Actually my gym costs $10 a month...,actually gym costs ten month way better equipm...
7574,1375572937739210754,@annabkrr Got my second shot &amp; I continue ...,0,@annabkrr Got my second shot &amp; I continue ...,got second shot amp continue wear maskeven ful...
7575,1387648123246350338,Went to hospital yesterday because I had some ...,1,Went to hospital yesterday because I had some ...,went hospital yesterday chest pain got chest x...
7576,1387906108514136066,Well... I got my second Pfizer dose two weeken...,0,Well... I got my second Pfizer dose two weeken...,well got second pfizer dose two weekends ago r...
7577,1304095108862586880,"last night, after reading about the 46-year-ol...",0,"last night, after reading about the 46-year-ol...",last night reading fortysixyearold woman diagn...
7578,1373826228105383937,"And a few weeks ago, I had COVID (very mild th...",1,"And a few weeks ago, I had COVID (very mild th...",weeks ago covid mild thankfully wife found got...
7579,1349069842377187328,Seeing all these Members of Congress (who’ve a...,0,Seeing all these Members of Congress (who’ve a...,seeing members congress received first dose va...


# Feature Extraction: TF-IDF (char_wb)

In [41]:
Tfidf_vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(1, 5), max_df=1.0, min_df=1, max_features=5000)
count_train = Tfidf_vec.fit(X_train)
train_features1 = Tfidf_vec.transform(X_train)
test_features1 = Tfidf_vec.transform(X_test)

Feature Extraction- (word) TFIDF

In [42]:
 Tfidf_vec1 = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), max_df=1.0, min_df=1, max_features=5000)
 count_train1 = Tfidf_vec1.fit(X_train)
 train_features2= Tfidf_vec1.transform(X_train)
 test_features2 = Tfidf_vec1.transform(X_test)

In [43]:
Tfidf_vec.get_feature_names()

C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[' ',
 ' a',
 ' ab',
 ' abl',
 ' able',
 ' ac',
 ' acc',
 ' act',
 ' actu',
 ' ad',
 ' adm',
 ' admi',
 ' af',
 ' ag',
 ' age',
 ' ago',
 ' ago ',
 ' al',
 ' all',
 ' alm',
 ' almo',
 ' alo',
 ' alon',
 ' alr',
 ' alre',
 ' als',
 ' also',
 ' am',
 ' ame',
 ' amp',
 ' amp ',
 ' an',
 ' ann',
 ' ano',
 ' anot',
 ' ant',
 ' anti',
 ' any',
 ' anyo',
 ' anyt',
 ' ap',
 ' app',
 ' apr',
 ' apri',
 ' ar',
 ' aro',
 ' arou',
 ' as',
 ' ask',
 ' ass',
 ' ast',
 ' at',
 ' att',
 ' au',
 ' av',
 ' aw',
 ' awa',
 ' away',
 ' b',
 ' ba',
 ' bac',
 ' back',
 ' bad',
 ' bad ',
 ' bas',
 ' be',
 ' bec',
 ' bed',
 ' bel',
 ' beli',
 ' bet',
 ' bett',
 ' bi',
 ' big',
 ' bit',
 ' bl',
 ' blo',
 ' bloo',
 ' bo',
 ' bod',
 ' br',
 ' bra',
 ' bre',
 ' brea',
 ' bri',
 ' bro',
 ' bu',
 ' c',
 ' ca',
 ' cal',
 ' call',
 ' cam',
 ' came',
 ' can',
 ' canc',
 ' car',
 ' care',
 ' cas',
 ' case',
 ' cat',
 ' catc',
 ' cau',
 ' caug',
 ' caus',
 ' ce',
 ' cer',
 ' cert',
 ' ch',
 ' cha',
 ' chan',
 ' che',
 ' 

# Model Building with SVM - LinearSVC-TF-IDF-char

In [44]:
clf1 =LinearSVC(C=1.0, class_weight="balanced", max_iter=10000, random_state=123)
clf1.fit(train_features1, y_train)
y_pred1=clf1.predict(test_features1)
accuracy = accuracy_score(y_test, y_pred1)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, y_pred1))

Test Accuracy: 79.1866

               precision    recall  f1-score   support

           0       0.92      0.81      0.86      2040
           1       0.46      0.70      0.56       468

    accuracy                           0.79      2508
   macro avg       0.69      0.76      0.71      2508
weighted avg       0.84      0.79      0.81      2508



# Model Building with SVM - LinearSVC-TF-IDF-word

In [45]:
clf1 =LinearSVC(C=1.0, class_weight="balanced", max_iter=10000, random_state=123)
clf1.fit(train_features2, y_train)
y_pred2=clf1.predict(test_features2)
accuracy = accuracy_score(y_test, y_pred2)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, y_pred2))

Test Accuracy: 81.2998

               precision    recall  f1-score   support

           0       0.91      0.86      0.88      2040
           1       0.50      0.62      0.55       468

    accuracy                           0.81      2508
   macro avg       0.70      0.74      0.72      2508
weighted avg       0.83      0.81      0.82      2508



# Feature Extraction: count vectorizer(char)

In [46]:
count_vec = CountVectorizer(analyzer='char_wb', ngram_range=(1, 5), max_df=1.0, min_df=1, max_features=5000)
count_train = count_vec.fit(X_train)
train_features3 = count_vec.transform(X_train)
test_features3 = count_vec.transform(X_test)

# Feature Extraction: count vectorizer(word)

In [47]:
count_vec1 = CountVectorizer(analyzer='word', ngram_range=(1, 3), max_df=1.0, min_df=1, max_features=5000)
count_train = count_vec1.fit(X_train)
train_features4 = count_vec1.transform(X_train)
test_features4 = count_vec1.transform(X_test)

# Model Building with SVM - LinearSVC-CountVectorizer-char

In [49]:
clf1 =LinearSVC(C=1.0, class_weight="balanced", max_iter=10000, random_state=123)
clf1.fit(train_features3, y_train)
y_pred1=clf1.predict(test_features3)
accuracy = accuracy_score(y_test, y_pred1)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, y_pred1))

Test Accuracy: 79.7448

               precision    recall  f1-score   support

           0       0.88      0.86      0.87      2040
           1       0.46      0.50      0.48       468

    accuracy                           0.80      2508
   macro avg       0.67      0.68      0.68      2508
weighted avg       0.80      0.80      0.80      2508



C:\Users\User\anaconda3\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# Model Building with SVM - LinearSVC-CountVectorizer-word

In [50]:
clf1 =LinearSVC(C=1.0, class_weight="balanced", max_iter=10000, random_state=123)
clf1.fit(train_features4, y_train)
y_pred2=clf1.predict(test_features4)
accuracy = accuracy_score(y_test, y_pred2)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, y_pred2))

Test Accuracy: 80.2632

               precision    recall  f1-score   support

           0       0.89      0.87      0.88      2040
           1       0.47      0.51      0.49       468

    accuracy                           0.80      2508
   macro avg       0.68      0.69      0.69      2508
weighted avg       0.81      0.80      0.81      2508

